In [1]:
import pandas as pd
import numpy as np
import aux.acessos as ac

In [2]:
query = """
        with perfil as (
select 
cpf
, ds_gender
, ds_region
, ds_occupation
, case
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 2 then 'a - ate 24 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 3 then 'b - 25-34 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 4 then 'c - 35-44 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 5 then 'd - 45-54 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 6 then 'e - 55-64 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 7 then 'f - 65-74 anos'
                when round((extract(day from current_date - cast(dt_birth as date))/30.5/12 / 10),0) = 8 then 'g - 75-84 anos'
                else 'h - idade NI' end as pf_faixa_idade    
from (
select
g.*, c.cpf
, rank() over (partition by c.cpf order by c.cd_yearmonth desc) as ordem_registro
from customer_curated_zone.ca_book_growth g 
left join customer_curated_zone.ca_book_cliente c on (g.id_customer = c.id_customer)
) where ordem_registro = 1
)
select *
from platform_sandbox_zone.flavia_sow_apos_churn a
left join perfil p on p.cpf = a.cpf 

"""

In [3]:
base = ac.df_athena('flavia-costa', query)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [4]:
len(base)

3562113

In [5]:
print(base.columns)

Index(['dt_mes_churn', 'cpf', 'utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3', 'cpf', 'ds_gender', 'ds_region',
       'ds_occupation', 'pf_faixa_idade'],
      dtype='object')


In [6]:
base_cluster = base[['utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3']]
base_cluster.head()

,utilizado_interno_m1,utilizado_mercado_m1,utilizado_interno_m2,utilizado_mercado_m2,utilizado_interno_m3,utilizado_mercado_m3
0,1629.77,7347.0,1629.77,5792.0,1629.77,3198.0
1,1098.78,10575.0,825.03,9595.0,929.60,9833.0
2,0.00,2753.0,0.00,2025.0,0.00,2199.0
3,0.00,2753.0,0.00,2025.0,0.00,2199.0
4,567.54,1128.0,588.92,1123.0,633.72,1118.0


In [7]:
for i in base_cluster.columns:
    base_cluster[i] = base_cluster[i].fillna(0.0)


<ipython-input-7-1f3d1368585a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_cluster[i] = base_cluster[i].fillna(0.0)


In [8]:
for i in base_cluster.columns:
    print(base_cluster[i].sum())
    

3836885953.259999
12454733271.0
3925942266.249995
12780501832.0
2614280501.0500016
8577043021.0


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [10]:
scaler = StandardScaler()
scaler_model = scaler.fit(base_cluster)
data_input_scaled = scaler_model.transform(base_cluster)

In [11]:
kmeans = KMeans(n_clusters = 5, random_state=0).fit(data_input_scaled)
cluster = kmeans.predict(data_input_scaled)
base['cluster'] = cluster

In [12]:
base['cluster'].value_counts(normalize = True)

1    0.781399
4    0.182515
0    0.018261
3    0.017155
2    0.000670
Name: cluster, dtype: float64

In [19]:
base['cluster'].value_counts()

1    2783430
4     650140
0      65048
3      61107
2       2388
Name: cluster, dtype: int64

In [13]:
var_cl = ['utilizado_interno_m1', 'utilizado_mercado_m1', 'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3', 'utilizado_mercado_m3']
base.groupby('cluster')[var_cl].mean().to_csv('cluster_final.csv')

In [15]:
infos = ['cluster', 'ds_gender', 'ds_region', 'ds_occupation', 'pf_faixa_idade']

base.groupby(infos).count().to_csv('cluster_final_perfis.csv')

In [20]:
base['flag_interno_m1'] = np.where(base['utilizado_interno_m1'] > 0, 1,0)
base['flag_interno_m2'] = np.where(base['utilizado_interno_m2'] > 0, 1,0)
base['flag_interno_m3'] = np.where(base['utilizado_interno_m3'] > 0, 1,0)
base['flag_mercado_m1'] = np.where(base['utilizado_mercado_m1'] > 0, 1,0)
base['flag_mercado_m2'] = np.where(base['utilizado_mercado_m2'] > 0, 1,0)
base['flag_mercado_m3'] = np.where(base['utilizado_mercado_m3'] > 0, 1,0)


In [16]:
base.query('utilizado_interno_m1 >  0').groupby('cluster')['utilizado_interno_m1', 'utilizado_mercado_m1'].mean()

<ipython-input-16-e622b9cc226a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.query('utilizado_interno_m1 >  0').groupby('cluster')['utilizado_interno_m1', 'utilizado_mercado_m1'].mean()


,utilizado_interno_m1,utilizado_mercado_m1
cluster,,
0,2099.507140,57276.669204
1,947.499179,3244.038312
2,4543.759101,280482.705545
3,9729.467602,10245.337966
4,2953.533069,4037.766297


In [22]:
base.groupby('cluster')['flag_interno_m1', 'flag_mercado_m1'].mean()

<ipython-input-22-d1b4a96b1396>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['flag_interno_m1', 'flag_mercado_m1'].mean()


,flag_interno_m1,flag_mercado_m1
cluster,,
0,0.377660,0.999908
1,0.483212,0.548433
2,0.438023,1.000000
3,0.998707,0.584025
4,0.995939,0.385017


In [17]:
base.query('utilizado_interno_m2 >  0').groupby('cluster')['utilizado_interno_m2', 'utilizado_mercado_m2'].mean()

<ipython-input-17-a05d8020bc37>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.query('utilizado_interno_m2 >  0').groupby('cluster')['utilizado_interno_m2', 'utilizado_mercado_m2'].mean()


,utilizado_interno_m2,utilizado_mercado_m2
cluster,,
0,2199.627120,59516.684373
1,955.889494,3327.937849
2,4779.563377,290478.023540
3,9963.953157,10386.580499
4,3002.054717,4148.150226


In [23]:
base.groupby('cluster')['flag_interno_m2', 'flag_mercado_m2'].mean()

<ipython-input-23-d1e534b0e33e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['flag_interno_m2', 'flag_mercado_m2'].mean()


,flag_interno_m2,flag_mercado_m2
cluster,,
0,0.388667,0.998939
1,0.491516,0.547522
2,0.445142,0.999581
3,0.999542,0.560607
4,0.998542,0.368211


In [18]:
base.query('utilizado_interno_m3 >  0').groupby('cluster')['utilizado_interno_m3', 'utilizado_mercado_m3'].mean()

<ipython-input-18-8fe829efa1e7>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.query('utilizado_interno_m3 >  0').groupby('cluster')['utilizado_interno_m3', 'utilizado_mercado_m3'].mean()


,utilizado_interno_m3,utilizado_mercado_m3
cluster,,
0,2153.331946,55846.663269
1,888.211025,3209.327729
2,4888.643014,272524.149888
3,9385.846865,9737.816276
4,2722.139872,3928.067495


In [24]:
base.groupby('cluster')['flag_interno_m3', 'flag_mercado_m3'].mean()

<ipython-input-24-ca8e7be3b76b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['flag_interno_m3', 'flag_mercado_m3'].mean()


,flag_interno_m3,flag_mercado_m3
cluster,,
0,0.304099,0.772460
1,0.304610,0.353146
2,0.376466,0.826214
3,0.852079,0.450390
4,0.748971,0.257661


In [25]:
base['utilizado_interno_total'] = base['utilizado_interno_m1'] + base['utilizado_interno_m2'] + base['utilizado_interno_m3']

base['utilizado_mercado_total'] = base['utilizado_mercado_m1'] + base['utilizado_mercado_m2'] + base['utilizado_mercado_m3']


In [27]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].mean()

<ipython-input-27-a6eacaa2b07d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].mean()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,2357.457117,155775.723538
1,1289.519043,11016.229850
2,6359.600465,844700.172326
3,27650.334158,29244.659960
4,8052.157862,11796.352352


In [29]:
import aux.aed as aed

In [30]:
valores_analise = ['utilizado_interno_total', 'utilizado_mercado_total']
aed.tira_outlier(base, valores_analise).groupby('cluster')[valores_analise].mean()

,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,5296.095810,157953.470880
1,2478.433053,9760.522661
2,7502.692693,799457.691267
3,21332.723257,21340.840406
4,8052.157862,11796.352352


In [26]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].median()

<ipython-input-26-3a8f7128bef2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].median()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,0.000,125970.0
1,55.885,5416.0
2,0.000,685460.0
3,23451.570,11831.0
4,7089.600,5829.0


In [31]:
base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].sum()

<ipython-input-31-0bf97582e43a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  base.groupby('cluster')['utilizado_interno_total', 'utilizado_mercado_total'].sum()


,utilizado_interno_total,utilizado_mercado_total
cluster,,
0,1.189054e+08,7.826484e+09
1,2.188855e+09,1.211021e+10
2,1.257929e+07,1.666593e+09
3,1.440555e+09,9.099184e+08
4,3.928310e+09,2.323469e+09


In [33]:
base.columns

Index(['dt_mes_churn', 'cpf', 'utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3', 'cpf', 'ds_gender', 'ds_region',
       'ds_occupation', 'pf_faixa_idade', 'cluster', 'flag_interno_m1',
       'flag_interno_m2', 'flag_interno_m3', 'flag_mercado_m1',
       'flag_mercado_m2', 'flag_mercado_m3', 'utilizado_interno_total',
       'utilizado_mercado_total'],
      dtype='object')

In [34]:
query2 = """
        
	select a.cpf
, max(ct.vl_credit_limit) as vl_credit_limit
from platform_sandbox_zone.flavia_sow_apos_churn a
left join customer_curated_zone.ca_book_cliente c on (a.cpf = c.cpf)
left join customer_curated_zone.ca_book_cartao ct on c.id_customer  = ct.id_customer 
group by 1

"""

In [35]:
limite = ac.df_athena('flavia-costa', query2)

In [41]:
limite.head()

,cpf,vl_credit_limit
0,07703542970,NaN
1,12098682450,NaN
2,42300568187,6100.0
3,04802781644,NaN
4,06371666550,NaN


In [46]:
base = base.loc[:,~base.columns.duplicated()]

In [47]:
base.columns

Index(['dt_mes_churn', 'cpf', 'utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3', 'ds_gender', 'ds_region', 'ds_occupation',
       'pf_faixa_idade', 'cluster', 'flag_interno_m1', 'flag_interno_m2',
       'flag_interno_m3', 'flag_mercado_m1', 'flag_mercado_m2',
       'flag_mercado_m3', 'utilizado_interno_total',
       'utilizado_mercado_total'],
      dtype='object')

In [48]:
base2 = base.join(limite.set_index('cpf'), on = 'cpf', how = 'left' )

In [50]:
base2.columns

Index(['dt_mes_churn', 'cpf', 'utilizado_interno_m1', 'utilizado_mercado_m1',
       'utilizado_interno_m2', 'utilizado_mercado_m2', 'utilizado_interno_m3',
       'utilizado_mercado_m3', 'ds_gender', 'ds_region', 'ds_occupation',
       'pf_faixa_idade', 'cluster', 'flag_interno_m1', 'flag_interno_m2',
       'flag_interno_m3', 'flag_mercado_m1', 'flag_mercado_m2',
       'flag_mercado_m3', 'utilizado_interno_total', 'utilizado_mercado_total',
       'vl_credit_limit'],
      dtype='object')

In [49]:
base2.groupby('cluster')['vl_credit_limit'].mean()

cluster
0    3782.358658
1    1853.910959
2    5351.428855
3    6991.746696
4    3129.037364
Name: vl_credit_limit, dtype: float64

In [32]:
base.to_pickle('base_clusters_share_of_wallet.pkl')